In [ ]:
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf
import sys

sys.path.append(r"../../")
from circe.models.LightningClassifier import LightningClassifier

cfg_model = OmegaConf.load('../training/conf/model/hf-gpt.yaml')
cfg_data = OmegaConf.load('../training/conf/data/vq-codebook.yaml')

ckpt_path = "../../../models-hfGPT-specvqgan/lightning_logs/version_27/checkpoints/epoch=144-step=580.ckpt"
model = LightningClassifier(cfg=cfg_model)
model.configure_sharded_model()
model.load_state_dict(torch.load(ckpt_path)["state_dict"])
model.eval()

In [ ]:
npy_file = "???"

In [ ]:
import numpy as np
from einops import rearrange
import random

values = torch.from_numpy(np.load(npy_file)).to(next(model.parameters()).device)
values = rearrange(values, "(f t) 1 -> f t", f=5)
rand_start = random.randint(0, values.shape[-1] - cfg_model.block_size // 5 - 2)
inputs = values[..., rand_start:rand_start + cfg_model.block_size // 5]
inputs = rearrange(inputs, "f t -> (t f)", f=5)
target = values[..., rand_start + 1:rand_start + cfg_model.block_size // 5 + 1]
target = rearrange(target, "f t -> (t f)", f=5)

with torch.inference_mode():
    print(inputs)
    logits = model(inputs)
    idx_next = torch.argmax(logits, dim=-1, keepdim=True)
    print(f"Cross entropy loss: {torch.nn.functional.cross_entropy(logits, target)}")
    print(f"Next indices: {idx_next}")
    print(f"Targets: {target}")